In [72]:
from pathlib import Path
import sqlite3
import pandas as pd
import sqlalchemy

In [73]:
Path('SXDB.db').touch()

In [74]:
conn = sqlite3.connect('SXDB.db')
c = conn.cursor()

In [75]:
df = pd.read_csv('Spacex.csv')
df.head()

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing _Outcome
0,04-06-2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,08-12-2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,22-05-2012,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,08-10-2012,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,01-03-2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [76]:
df.rename(columns={'Landing _Outcome':'Landing_Outcome'}, inplace=True)

In [77]:
df.head()

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,04-06-2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,08-12-2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,22-05-2012,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,08-10-2012,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,01-03-2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [113]:
df.columns

Index(['Date', 'Time (UTC)', 'Booster_Version', 'Launch_Site', 'Payload',
       'PAYLOAD_MASS__KG_', 'Orbit', 'Customer', 'Mission_Outcome',
       'Landing_Outcome'],
      dtype='object')

In [78]:
c.execute(''' CREATE TABLE IF NOT EXISTS SXDB (
                        'Date' DATE,
                        'Time (UTC)' DATETIME,
                        'Booster_Version' varchar(30),
                        'Launch_Site' varchar(30),
                        'Payload' vrchar(80),
                        'PAYLOAD_MASS__KG_' int(90),
                        'Orbit' varchar(30),
                        'Customer' varchar (60),
                        'Mission_Outcome' varchar(50),
                        'Landing_Outcome' varchar(80))''')

In [79]:
df.to_sql('SXDB', conn, if_exists='append', index=False)

/home/edward/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [80]:
c.execute('''SELECT * FROM SXDB''')

## Tasks

Now write and execute SQL queries to solve the assignment tasks.

### Task 1

##### Display the names of the unique launch sites  in the space mission


In [81]:
c.execute('''SELECT DISTINCT Launch_Site FROM SXDB''').fetchall()

[('CCAFS LC-40',), ('VAFB SLC-4E',), ('KSC LC-39A',), ('CCAFS SLC-40',)]

### Task 2

##### Display 5 records where launch sites begin with the string 'CCA'

In [82]:
c.execute('''SELECT * FROM SXDB WHERE Launch_Site > 'CCA' LIMIT 5''').fetchall()

[('04-06-2010',
  '18:45:00',
  'F9 v1.0  B0003',
  'CCAFS LC-40',
  'Dragon Spacecraft Qualification Unit',
  0,
  'LEO',
  'SpaceX',
  'Success',
  'Failure (parachute)'),
 ('08-12-2010',
  '15:43:00',
  'F9 v1.0  B0004',
  'CCAFS LC-40',
  'Dragon demo flight C1, two CubeSats, barrel of Brouere cheese',
  0,
  'LEO (ISS)',
  'NASA (COTS) NRO',
  'Success',
  'Failure (parachute)'),
 ('22-05-2012',
  '07:44:00',
  'F9 v1.0  B0005',
  'CCAFS LC-40',
  'Dragon demo flight C2',
  525,
  'LEO (ISS)',
  'NASA (COTS)',
  'Success',
  'No attempt'),
 ('08-10-2012',
  '00:35:00',
  'F9 v1.0  B0006',
  'CCAFS LC-40',
  'SpaceX CRS-1',
  500,
  'LEO (ISS)',
  'NASA (CRS)',
  'Success',
  'No attempt'),
 ('01-03-2013',
  '15:10:00',
  'F9 v1.0  B0007',
  'CCAFS LC-40',
  'SpaceX CRS-2',
  677,
  'LEO (ISS)',
  'NASA (CRS)',
  'Success',
  'No attempt')]

### Task 3

##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [83]:
c.execute('''SELECT SUM(PAYLOAD_MASS__KG_) FROM SXDB WHERE Customer = "NASA (CRS)"''').fetchall()

[(45596,)]

### Task 4

##### Display average payload mass carried by booster version F9 v1.1


In [84]:
c.execute('''SELECT AVG(PAYLOAD_MASS__KG_) FROM SXDB WHERE Booster_Version = "F9 v1.1"''').fetchall()

[(2928.4,)]

### Task 5

##### List the date when the first successful landing outcome in ground pad was acheived.

*Hint:Use min function*


In [85]:
c.execute(''' SELECT MIN(Date) FROM SXDB WHERE Mission_Outcome ="Success" ''').fetchall()

[('01-03-2013',)]

### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [102]:
c.execute(''' SELECT Booster_Version FROM SXDB WHERE Landing_Outcome = "Success (drone ship)" AND PAYLOAD_MASS__KG_ BETWEEN 4000 AND 6000 ''').fetchall()

[('F9 FT B1022',), ('F9 FT B1026',), ('F9 FT  B1021.2',), ('F9 FT  B1031.2',)]

### Task 7

##### List the total number of successful and failure mission outcomes


In [103]:
c.execute('''SELECT COUNT(*), Mission_Outcome FROM SXDB GROUP BY Mission_Outcome''').fetchall()

[(1, 'Failure (in flight)'),
 (98, 'Success'),
 (1, 'Success '),
 (1, 'Success (payload status unclear)')]

### Task 8

##### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [104]:
c.execute('''SELECT Booster_Version FROM SXDB WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM SXDB)''').fetchall()

[('F9 B5 B1048.4',),
 ('F9 B5 B1049.4',),
 ('F9 B5 B1051.3',),
 ('F9 B5 B1056.4',),
 ('F9 B5 B1048.5',),
 ('F9 B5 B1051.4',),
 ('F9 B5 B1049.5',),
 ('F9 B5 B1060.2 ',),
 ('F9 B5 B1058.3 ',),
 ('F9 B5 B1051.6',),
 ('F9 B5 B1060.3',),
 ('F9 B5 B1049.7 ',)]

### Task 9

##### List the failed landing_outcomes in drone ship, their booster versions, and launch site names for in year 2015


In [120]:
c.execute('''SELECT Booster_Version, Launch_Site FROM SXDB WHERE Landing_Outcome = "Failure (drone ship)" AND Date BETWEEN "01-01-2015" AND "31-12-2015"''').fetchall()

[('F9 v1.1 B1012', 'CCAFS LC-40'),
 ('F9 v1.1 B1015', 'CCAFS LC-40'),
 ('F9 v1.1 B1017', 'VAFB SLC-4E'),
 ('F9 FT B1020', 'CCAFS LC-40'),
 ('F9 FT B1024', 'CCAFS LC-40')]

### Task 10

##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order


In [121]:
c.execute(''' SELECT COUNT(*) AS count, Landing_Outcome FROM SXDB WHERE Date BETWEEN "04-06-2010" AND "20-03-2017" GROUP BY Landing_Outcome ORDER BY count DESC''').fetchall()

[(20, 'Success'),
 (10, 'No attempt'),
 (8, 'Success (drone ship)'),
 (6, 'Success (ground pad)'),
 (4, 'Failure (drone ship)'),
 (3, 'Failure'),
 (3, 'Controlled (ocean)'),
 (2, 'Failure (parachute)'),
 (1, 'No attempt ')]